In [ ]:
you have two tables 'invoices' and ' users':
- invoices: one row per invoice sent, consists of the following columns:
	- invoice_id: unique invoice identifier
	- user_id: uniquer sender identifier
	- sent_at: datetime invoice was sent
	- amount: invoice amount
- users: one row per user. consists of the following columns:
	- user_id: unique user identifier
	- created_at: user create timestamp
	- business_category: business category (e.g. beauty_and_personal_care)

Q1: who are the top 5 invoice senders by amount?

Q2: what are the total senders, counts of invoice, and amounts sent by business_category?

Q3: How many sellers are in the invoices table but not in user table?

Q4: How many sellers are in the users table but not in invoices table?

Q5: How many users sent more than one invoice?

Q6: What are the details of the most recent invoices of each user?

Q7: What is the distribution of invoices sent per user? i.e. how many users sent 0,1,2,3...invoices.

In [ ]:
#Q1: who are the top 5 invoice senders by amount?

In [ ]:
SELECT *
FROM invoices
ORDER BY amount DESC
LIMIT 5

In [ ]:
#Q2: what are the total senders, counts of invoice, and amounts sent by business_category?

In [ ]:
SELECT COUNT(DISTINCT sub.user_id), COUNT(sub.invoice_id), SUM(sub.amount)
FROM(
SELECT * FROM invoices AS i
OUTER JOIN users AS u
ON i.user_id = u.user_id) AS sub

In [ ]:
SELECT SUM(sub.amount) AS Amount_sent_by_beauty_and_personal_care 
FROM(
SELECT * FROM invoices AS i
OUTER JOIN users AS u
ON i.user_id = u.user_id) AS sub
WHERE sub.business_category = 'beauty_and_personal_care'

In [ ]:
#Q3: How many sellers are in the invoices table but not in user table?

In [ ]:
SELECT COUNT(sub.user_id)
FROM(
SELECT * FROM invoices AS i
LEFT JOIN users AS u
ON i.user_id = u.user_id
WHERE u.user_id IS NULL) AS sub

In [ ]:
#Q4: How many sellers are in the users table but not in invoices table?

In [ ]:
SELECT COUNT(sub.user_id)
FROM(
SELECT * FROM invoices AS i
RIGHT JOIN users AS u
ON i.user_id = u.user_id
WHERE i.user_id IS NULL) AS sub

In [ ]:
#Q5: How many users sent more than one invoice?

In [ ]:
SELECT *, COUNT(user_id) AS Count
FROM invoices
GROUP BY user_id
HAVING COUNT(*) > 1

In [ ]:
#Q6: What are the details of the most recent invoices of each user?

In [ ]:
#see who is the most recent
SELECT * FROM invoices AS i
LEFT JOIN users AS u
ON i.user_id = u.user_id

ORDER BY sent_at, user_id DESC

In [ ]:
#see the most recent amount
SELECT * FROM invoices AS i
LEFT JOIN users AS u
ON i.user_id = u.user_id

ORDER BY sent_at, user_id, amount DESC

In [ ]:
#see the most recent amount of 'beauty_and_personal_care'
SELECT * FROM invoices AS i
LEFT JOIN users AS u
ON i.user_id = u.user_id
WHERE u.business_category = 'beauty_and_personal_care'
ORDER BY sent_at, user_id, amount  DESC

In [ ]:
#see the most recent amount of active?
SELECT *, (sent_at-created_at) AS active_days FROM invoices AS i
LEFT JOIN users AS u
ON i.user_id = u.user_id
ORDER BY sent_at, user_id, created_at DESC

In [ ]:
#Q7: What is the distribution of invoices sent per user? i.e. how many users sent 0,1,2,3...invoices.

In [ ]:
#see how many users sent 0,1,2 invoices
create table temp AS
SELECT Count1 AS Num_of_invoices, COUNT(Count1) AS user_Count, 
FROM(
SELECT user_id, COUNT(user_id) AS Count1 FROM invoices AS i
LEFT JOIN users AS u
ON i.user_id = u.user_id
GROUP BY user_id)
GROUP BY Count1


SELECT *, user_Count * 100/(SELECT SUM(user_Count) FROM temp) as 'Percentage of Total(%)'
FROM temp

In [ ]:
#see how many users sent 0,1,2 based on business_category
create table temp_business_category AS
SELECT business_category, COUNT(business_category) AS Count1 FROM invoices
LEFT JOIN users AS u
ON i.user_id = u.user_id
GROUP BY business_category


SELECT *, Count1 * 100/(SELECT SUM(Count1) FROM temp1) as 'Percentage of Total(%)'
FROM temp_business_category

In [ ]:
# based on amount group
create table temp_amount AS
select t.range as [score range], count(*) as Count1
from (
      select *,
         case when amount >= 0 and amount< 500 then '0-500'
         when amount >= 500 and amount< 1000 then '500-1000'
         else '1000<=' end as range
     from invoices
     LEFT JOIN users AS u
     ON i.user_id = u.user_id) t
group by t.range



SELECT *, Count1 * 100/(SELECT SUM(Count1) FROM temp1) as 'Percentage of Total(%)'
FROM temp_amount